In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score

from xgboost import XGBClassifier

from splits import split_users # contains split_users func

pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 100)

In [2]:
# # read in pickled DFs generated by query_dfs.py

# df_orders = pd.read_pickle("./pickle/df_orders.pickle")
# df_train = pd.read_pickle("./pickle/df_train.pickle")
# df_prior = pd.read_pickle("./pickle/df_prior.pickle")
# df_prod_detail = pd.read_pickle("./pickle/df_prod_detail.pickle")

In [3]:
# read in pickled feature DF generated by feature_engineering_1.ipynb
X = pd.read_pickle("./pickle/X_25.pickle")

In [4]:
# check scikit-learn version
import sklearn
print(sklearn.__version__)

0.24.1


In [5]:
# X_train, X_test, y_train, y_test = split_users(X, subset=0.5, test_size=0.2)# only using 10%


In [6]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

# rf = RandomForestClassifier(n_estimators=2000, max_depth=8, max_features=8, n_jobs=-1)
# rf_fit = rf.fit(X_train, y_train)


In [7]:
# preds = rf_fit.predict(X_test)

In [8]:
# f1_score(y_test, preds)

In [9]:
# with open(f"models/rf_fit.pickle", "wb") as pfile:
#         pickle.dump(rf_fit, pfile)

In [10]:


# X_train, X_test, y_train, y_test = split_users(X, subset=0.2, test_size=0.2) # only using 10%

# xgb = XGBClassifier(n_estimators=300, eval_metric='logloss', use_label_encoder =False)
# xgb_fit = xgb.fit(X_train, y_train)

In [11]:
# with open(f"models/xgboost_fit.pickle", "wb") as pfile:
#         pickle.dump(xgb_fit, pfile)

In [12]:
# xgb_fit.score(X_test, y_test)

In [13]:
# xgb_preds = xgb_fit.predict(X_test)

In [14]:
# f1_score(y_test, xgb_preds)

## Grid Search

In [25]:
X_train, X_test, y_train, y_test = split_users(X, subset=0.02, test_size=0.2) # only using 10%



    X_train sample size: 185180
    X_test sample size: 42450


In [26]:
estimator = XGBClassifier(objective='binary:logistic',
                          use_label_encoder=False,
                          eval_metric='logloss',
                          random_state=54,
)

params = {
    'max_depth': range(2, 10, 1),
    'n_estimators': range(50, 650, 150),
    'learning_rate': [0.01, 0.05, 0.1]
}

In [27]:
grid_search = GridSearchCV(
    estimator = estimator,
    param_grid = params
)

In [28]:
grid_xgb = grid_search.fit(X_train, y_train)
print("The best parameters are: \n", grid_search.best_params_)



The best parameters are: 
 {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}


In [29]:
with open(f"models/grid_xgb.pickle", "wb") as pfile:
        pickle.dump(grid_xgb, pfile)

Great. Now we have our parameters. Let's run the model!

In [31]:
# X_train, X_test, y_train, y_test = split_users(X, subset=0.5, test_size=0.2) # only using 10%


    X_train sample size: 4218232
    X_test sample size: 1055268


In [32]:
%%time
xgb = XGBClassifier(n_estimators=500,
                    max_depth=3,
                    learning_rate=0.01,
                    eval_metric='logloss',
                    use_label_encoder =False)
xgb_fit = xgb.fit(X_train, y_train)

CPU times: user 1h 12min 9s, sys: 860 ms, total: 1h 12min 10s
Wall time: 10min 9s


In [34]:
preds = xgb_fit.predict(X_test)

In [36]:
f1_score(y_test, preds)

0.23481636614394438